In [1]:
# ==============================================
# Import Necessary Libraries
# ==============================================
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.optimize import minimize
from sklearn.impute import KNNImputer

from scipy.stats import mode  # For majority voting

# PyTorch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Suppress warnings and set display options
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

# Set random seed for reproducibility
SEED = 0
np.random.seed(SEED)
torch.manual_seed(SEED)

In [2]:
# def process_file(filename, dirname):
#     df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
#     df.drop('step', axis=1, inplace=True)
#     return df.describe().values.reshape(-1), filename.split('=')[1]

# def load_time_series(dirname) -> pd.DataFrame:
#     ids = os.listdir(dirname)
    
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
#     stats, indexes = zip(*results)
    
#     df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
#     df['id'] = indexes
#     return df


# class AutoEncoder(nn.Module):
#     def __init__(self, input_dim, encoding_dim):
#         super(AutoEncoder, self).__init__()
#         self.encoder = nn.Sequential(
#             nn.Linear(input_dim, encoding_dim*3),
#             nn.ReLU(),
#             nn.Linear(encoding_dim*3, encoding_dim*2),
#             nn.ReLU(),
#             nn.Linear(encoding_dim*2, encoding_dim),
#             nn.ReLU()
#         )
#         self.decoder = nn.Sequential(
#             nn.Linear(encoding_dim, input_dim*2),
#             nn.ReLU(),
#             nn.Linear(input_dim*2, input_dim*3),
#             nn.ReLU(),
#             nn.Linear(input_dim*3, input_dim),
#             nn.Sigmoid()
#         )
        
#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
#         return decoded


# def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
#     scaler = StandardScaler()
#     df_scaled = scaler.fit_transform(df)
    
#     data_tensor = torch.FloatTensor(df_scaled)
    
#     input_dim = data_tensor.shape[1]
#     autoencoder = AutoEncoder(input_dim, encoding_dim)
    
#     criterion = nn.MSELoss()
#     optimizer = optim.Adam(autoencoder.parameters())
    
#     for epoch in range(epochs):
#         for i in range(0, len(data_tensor), batch_size):
#             batch = data_tensor[i : i + batch_size]
#             optimizer.zero_grad()
#             reconstructed = autoencoder(batch)
#             loss = criterion(reconstructed, batch)
#             loss.backward()
#             optimizer.step()
            
#         if (epoch + 1) % 10 == 0:
#             print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')
                 
#     with torch.no_grad():
#         encoded_data = autoencoder.encoder(data_tensor).numpy()
        
#     df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
#     return df_encoded

# def feature_engineering(df):
#     season_cols = [col for col in df.columns if 'Season' in col]
#     df = df.drop(season_cols, axis=1) 
#     df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
#     df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
#     df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
#     df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
#     df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
#     df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
#     df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
#     df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
#     df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
#     df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
#     df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
#     df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
#     df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
#     df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
#     df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    
#     return df

In [3]:
# train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
# test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
# sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
# test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# df_train = train_ts.drop('id', axis=1)
# df_test = test_ts.drop('id', axis=1)

# train_ts_encoded = perform_autoencoder(df_train, encoding_dim=60, epochs=100, batch_size=32)
# test_ts_encoded = perform_autoencoder(df_test, encoding_dim=60, epochs=100, batch_size=32)

# time_series_cols = train_ts_encoded.columns.tolist()
# train_ts_encoded["id"]=train_ts["id"]
# test_ts_encoded['id']=test_ts["id"]

# train = pd.merge(train, train_ts_encoded, how="left", on='id')
# test = pd.merge(test, test_ts_encoded, how="left", on='id')

# imputer = KNNImputer(n_neighbors=5)
# numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
# imputed_data = imputer.fit_transform(train[numeric_cols])
# train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)
# train_imputed['sii'] = train_imputed['sii'].round().astype(int)
# for col in train.columns:
#     if col not in numeric_cols:
#         train_imputed[col] = train[col]
        
# train = train_imputed

# train = feature_engineering(train)
# train = train.dropna(thresh=10, axis=0)
# test = feature_engineering(test)

# train = train.drop('id', axis=1)
# test  = test .drop('id', axis=1)   


# featuresCols = ['Basic_Demos-Age', 'Basic_Demos-Sex',
#                 'CGAS-CGAS_Score', 'Physical-BMI',
#                 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
#                 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
#                 'Fitness_Endurance-Max_Stage',
#                 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
#                 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
#                 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
#                 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
#                 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
#                 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
#                 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
#                 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
#                 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
#                 'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total',
#                 'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw',
#                 'SDS-SDS_Total_T',
#                 'PreInt_EduHx-computerinternet_hoursday', 'sii', 'BMI_Age','Internet_Hours_Age','BMI_Internet_Hours',
#                 'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW', 'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight',
#                 'SMM_Height', 'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW']

# featuresCols += time_series_cols

# train = train[featuresCols]
# train = train.dropna(subset='sii')

# featuresCols = ['Basic_Demos-Age', 'Basic_Demos-Sex',
#                 'CGAS-CGAS_Score', 'Physical-BMI',
#                 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
#                 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
#                 'Fitness_Endurance-Max_Stage',
#                 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
#                 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
#                 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
#                 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
#                 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
#                 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
#                 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
#                 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
#                 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
#                 'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total',
#                 'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw',
#                 'SDS-SDS_Total_T',
#                 'PreInt_EduHx-computerinternet_hoursday', 'BMI_Age','Internet_Hours_Age','BMI_Internet_Hours',
#                 'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW', 'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight',
#                 'SMM_Height', 'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW']

# featuresCols += time_series_cols
# test = test[featuresCols]

In [4]:
# test_df = test
# train_df = train

In [5]:
# ==============================================
# Load and Merge Data
# ==============================================
# Load main datasets
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

# ==============================================
# Feature Engineering
# ==============================================
def engineer_features(df):
    """
    Create interaction features.
    """
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['Pulse_Pressure'] = df['Physical-Systolic_BP'] - df['Physical-Diastolic_BP']
    df['HeartRate_Age'] = df['Physical-HeartRate'] * df['Basic_Demos-Age']
    df['Fitness_Score'] = df['Fitness_Endurance-Max_Stage'] * (
        df['Fitness_Endurance-Time_Mins'] * 60 + df['Fitness_Endurance-Time_Sec']
    )
    df['FMI_FFMI_Ratio'] = df['BIA-BIA_FMI'] / (df['BIA-BIA_FFMI'] + 1e-6)
    df['Sleep_Internet_Hours'] = df['SDS-SDS_Total_T'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['Waist_Height_Ratio'] = df['Physical-Waist_Circumference'] / (df['Physical-Height'] + 1e-6)
    return df

# Apply feature engineering to both train and test datasets
train = engineer_features(train)
test = engineer_features(test)

# ==============================================
# Define Feature Columns
# ==============================================
featuresCols = list(set(train.columns).intersection(set(test.columns)))
# featuresCols = [
#     'Internet_Hours_Age', 'Physical-Height', 'Sleep_Internet_Hours', 'Basic_Demos-Age', 
#     'BMI_Internet_Hours', 'HeartRate_Age', 'BMI_Age', 'Physical-Waist_Circumference', 
#     'FGC-FGC_CU', 'BIA-BIA_BMI', 'SDS-SDS_Total_Raw', 'FGC-FGC_GSND', 'Physical-BMI', 
#     'FGC-FGC_GSD', 'FGC-FGC_PU', 'BIA-BIA_FFMI', 'BIA-BIA_Frame_num', 'Physical-Systolic_BP', 
#     'Pulse_Pressure', 'FGC-FGC_TL', 'Basic_Demos-Sex', 'FGC-FGC_SRL_Zone', 'CGAS-CGAS_Score', 
#     'Fitness_Score', 'BIA-BIA_SMM', 'BIA-BIA_Activity_Level_num'
# ]

# Select the features from train and test datasets
train_df = train[featuresCols + ['sii']]  # Include 'sii' in train features
test_df = test[featuresCols]

# Drop 'id' from train and test data if present
train_df = train_df.drop('id', axis=1, errors='ignore')
test_df = test_df.drop('id', axis=1, errors='ignore')

# Drop rows with missing target variable 'sii' in train data
train_df = train_df.dropna(subset=['sii'])

featuresCols = list(test_df.columns)

# ==============================================
# Handle Categorical Variables
# ==============================================
# List of categorical columns
cat_c = [
    'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
    'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
    'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season'
]

def create_consistent_mapping(column):
    """
    Create a consistent mapping for categorical variables by combining unique values from both datasets.
    """
    combined_values = pd.concat([train_df[column], test_df[column]], axis=0).fillna('Missing')
    unique_values = combined_values.unique()
    sorted_values = np.sort(unique_values)  # Ensure consistent ordering
    return {value: idx for idx, value in enumerate(sorted_values)}

# Map categorical variables to integers using a consistent mapping
for col in cat_c:
    mapping = create_consistent_mapping(col)
    train_df[col] = train_df[col].fillna('Missing').map(mapping).astype(int)
    test_df[col] = test_df[col].fillna('Missing').map(mapping).astype(int)


# ==============================================
# Handle Missing Values with KNN Imputer
# ==============================================
# Impute missing values using KNNImputer
imputer = KNNImputer(n_neighbors=3)
numeric_cols = train_df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Remove 'sii' from numeric_cols when applying to test data
if 'sii' in numeric_cols:
    numeric_cols.remove('sii')

# Impute train data
train_df[numeric_cols] = imputer.fit_transform(train_df[numeric_cols])

# Impute test data
# Ensure that all columns in numeric_cols exist in test data
numeric_cols_test = [col for col in numeric_cols if col in test_df.columns]
test_df[numeric_cols_test] = imputer.transform(test_df[numeric_cols_test])

# Ensure 'sii' remains integer
train_df['sii'] = train_df['sii'].round().astype(int)

In [6]:
# ==============================================
# Model Training and Evaluation Functions
# ==============================================
def quadratic_weighted_kappa(y_true, y_pred):
    """
    Calculate Quadratic Weighted Kappa.
    """
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    """
    Apply thresholds to continuous predictions.
    """
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    """
    Objective function for optimizing thresholds.
    """
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

# ==============================================
# Split Features Among Expert Networks
# ==============================================
# feature_splits = np.array_split(featuresCols, 4)
# features_expert1 = feature_splits[0].tolist()
# features_expert2 = feature_splits[1].tolist()
# features_expert3 = feature_splits[2].tolist()
# features_expert4 = feature_splits[3].tolist()
features_expert1 = featuresCols
features_expert2 = featuresCols
features_expert3 = featuresCols
features_expert4 = featuresCols

# ==============================================
# Prepare Data for Training
# ==============================================
# Inputs for expert networks
X_expert1 = train_df[features_expert1]
X_expert2 = train_df[features_expert2]
X_expert3 = train_df[features_expert3]
X_expert4 = train_df[features_expert4]

# Input for weighting network
X_weighting = train_df[featuresCols]  # All features

# Target variable
y = train_df['sii']

# Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_index, val_index = next(sss.split(X_weighting, y))

X_expert_list_train = [X.iloc[train_index].reset_index(drop=True) for X in [X_expert1, X_expert2, X_expert3, X_expert4]]
X_weighting_train = X_weighting.iloc[train_index].reset_index(drop=True)
y_train = y.iloc[train_index].reset_index(drop=True)

X_expert_list_val = [X.iloc[val_index].reset_index(drop=True) for X in [X_expert1, X_expert2, X_expert3, X_expert4]]
X_weighting_val = X_weighting.iloc[val_index].reset_index(drop=True)
y_val = y.iloc[val_index].reset_index(drop=True)

# ==============================================
# Define Custom Dataset Class
# ==============================================
class MultiInputDataset(Dataset):
    def __init__(self, X_expert_list, X_weighting, y=None):
        self.X_expert_list = [torch.tensor(X.values, dtype=torch.float32) for X in X_expert_list]
        self.X_weighting = torch.tensor(X_weighting.values, dtype=torch.float32)
        if y is not None:
            self.y = torch.tensor(y.values, dtype=torch.float32)
        else:
            self.y = None
    def __len__(self):
        return len(self.X_weighting)
    def __getitem__(self, idx):
        inputs = [X[idx] for X in self.X_expert_list]
        weighting_input = self.X_weighting[idx]
        if self.y is not None:
            return inputs, weighting_input, self.y[idx]
        else:
            return inputs, weighting_input

# Create datasets and data loaders
train_dataset = MultiInputDataset(X_expert_list_train, X_weighting_train, y_train)
val_dataset = MultiInputDataset(X_expert_list_val, X_weighting_val, y_val)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Prepare test data
X_expert1_test = test_df[features_expert1]
X_expert2_test = test_df[features_expert2]
X_expert3_test = test_df[features_expert3]
X_expert4_test = test_df[features_expert4]
X_weighting_test = test_df[featuresCols]

test_dataset = MultiInputDataset(
    [X_expert1_test.reset_index(drop=True), X_expert2_test.reset_index(drop=True), 
     X_expert3_test.reset_index(drop=True), X_expert4_test.reset_index(drop=True)],
    X_weighting_test.reset_index(drop=True)
)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ==============================================
# Define Neural Network Architectures
# ==============================================
class ExpertNet(nn.Module):
    def __init__(self, input_size):
        super(ExpertNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.fc(x).squeeze()  # Output shape: [batch_size]

class WeightingNet(nn.Module):
    def __init__(self, input_size):
        super(WeightingNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32, 4),
            nn.Softmax(dim=1)  # Output weights that sum to 1
        )
    def forward(self, x):
        return self.fc(x)  # Output shape: [batch_size, 4]

# Initialize networks
input_sizes = [len(features_expert1), len(features_expert2), len(features_expert3), len(features_expert4)]
expert_nets = [ExpertNet(input_size) for input_size in input_sizes]
weighting_net = WeightingNet(len(featuresCols))

# Move models to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for net in expert_nets:
    net.to(device)
weighting_net.to(device)

# ==============================================
# Define Loss Function and Optimizer
# ==============================================
class OrdinalLoss(nn.Module):
    def __init__(self):
        super(OrdinalLoss, self).__init__()
    def forward(self, preds, targets):
        # preds: continuous predictions [batch_size]
        # targets: true labels [batch_size]
        # Compute loss that penalizes deviations more severely for larger errors
        loss = torch.mean((preds - targets.float()) ** 2)
        return loss

criterion = OrdinalLoss()

optimizer = optim.Adam(
    list(expert_nets[0].parameters()) +
    list(expert_nets[1].parameters()) +
    list(expert_nets[2].parameters()) +
    list(expert_nets[3].parameters()) +
    list(weighting_net.parameters()),
    lr=0.001
)

# ==============================================
# Training Loop
# ==============================================
num_epochs = 38

for epoch in range(num_epochs):
    expert_nets[0].train()
    expert_nets[1].train()
    expert_nets[2].train()
    expert_nets[3].train()
    weighting_net.train()
    
    running_loss = 0.0
    for inputs_list, weighting_input, labels in train_loader:
        optimizer.zero_grad()
        
        # Move data to device
        inputs_list = [inp.to(device) for inp in inputs_list]
        weighting_input = weighting_input.to(device)
        labels = labels.to(device)
        
        # Get outputs from expert networks
        expert_outputs = []
        for i in range(4):
            expert_outputs.append(expert_nets[i](inputs_list[i]))
        expert_outputs = torch.stack(expert_outputs, dim=1)  # Shape: [batch_size, 4]
        
        # Get weights from weighting network
        weights = weighting_net(weighting_input)  # Shape: [batch_size, 4]
        
        # Compute weighted sum
        preds = (expert_outputs * weights).sum(dim=1)  # Shape: [batch_size]
        
        # Compute loss
        loss = criterion(preds, labels)
        
        # Backpropagate
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * labels.size(0)
    epoch_loss = running_loss / len(train_dataset)
    
    # Validation
    expert_nets[0].eval()
    expert_nets[1].eval()
    expert_nets[2].eval()
    expert_nets[3].eval()
    weighting_net.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for inputs_list, weighting_input, labels in val_loader:
            # Move data to device
            inputs_list = [inp.to(device) for inp in inputs_list]
            weighting_input = weighting_input.to(device)
            labels = labels.to(device)
            
            # Get outputs from expert networks
            expert_outputs = []
            for i in range(4):
                expert_outputs.append(expert_nets[i](inputs_list[i]))
            expert_outputs = torch.stack(expert_outputs, dim=1)  # Shape: [batch_size, 4]
            
            # Get weights from weighting network
            weights = weighting_net(weighting_input)  # Shape: [batch_size, 4]
            
            # Compute weighted sum
            preds = (expert_outputs * weights).sum(dim=1)  # Shape: [batch_size]
            
            val_preds.append(preds.cpu().numpy())
            val_labels.append(labels.cpu().numpy())
    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    
    # Compute validation loss and QWK
    val_loss = np.mean((val_preds - val_labels) ** 2)
    val_preds_rounded = np.clip(np.round(val_preds), 0, 3).astype(int)
    val_kappa = quadratic_weighted_kappa(val_labels, val_preds_rounded)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val QWK: {val_kappa:.4f}")

# ==============================================
# Optimize Thresholds
# ==============================================
from scipy.optimize import minimize

# Optimize thresholds
KappaOptimizer = minimize(evaluate_predictions,
                          x0=[0.5, 1.5, 2.5], args=(val_labels, val_preds), 
                          method='Nelder-Mead')
best_thresholds = KappaOptimizer.x

# Evaluate optimized QWK
val_preds_tuned = threshold_Rounder(val_preds, best_thresholds)
val_kappa_tuned = quadratic_weighted_kappa(val_labels, val_preds_tuned)
print(f"Optimized Validation QWK: {val_kappa_tuned:.4f}")

# ==============================================
# Generate Predictions on Test Data
# ==============================================
expert_nets[0].eval()
expert_nets[1].eval()
expert_nets[2].eval()
expert_nets[3].eval()
weighting_net.eval()

test_preds = []
with torch.no_grad():
    for inputs_list, weighting_input in test_loader:
        # Move data to device
        inputs_list = [inp.to(device) for inp in inputs_list]
        weighting_input = weighting_input.to(device)
        
        # Get outputs from expert networks
        expert_outputs = []
        for i in range(4):
            expert_outputs.append(expert_nets[i](inputs_list[i]))
        expert_outputs = torch.stack(expert_outputs, dim=1)  # Shape: [batch_size, 4]
        
        # Get weights from weighting network
        weights = weighting_net(weighting_input)  # Shape: [batch_size, 4]
        
        # Compute weighted sum
        preds = (expert_outputs * weights).sum(dim=1)  # Shape: [batch_size]
        
        test_preds.append(preds.cpu().numpy())
test_preds = np.concatenate(test_preds)

# Apply optimized thresholds
test_preds_tuned = threshold_Rounder(test_preds, best_thresholds)
test_preds_tuned = test_preds_tuned.astype(int)

# Create submission DataFrame
submission4 = pd.DataFrame({
    'id': test['id'],
    'sii': test_preds_tuned
})

# Save submission
submission4.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' has been created.")

Epoch 1/38, Loss: 0.6123, Val Loss: 0.5432, Val QWK: 0.2895
Epoch 2/38, Loss: 0.4945, Val Loss: 0.5596, Val QWK: 0.2943
Epoch 3/38, Loss: 0.4925, Val Loss: 0.5034, Val QWK: 0.3438
Epoch 4/38, Loss: 0.4955, Val Loss: 0.4675, Val QWK: 0.3503
Epoch 5/38, Loss: 0.4714, Val Loss: 0.4941, Val QWK: 0.3155
Epoch 6/38, Loss: 0.4665, Val Loss: 0.5538, Val QWK: 0.2701
Epoch 7/38, Loss: 0.4779, Val Loss: 0.4978, Val QWK: 0.3264
Epoch 8/38, Loss: 0.4622, Val Loss: 0.4804, Val QWK: 0.3232
Epoch 9/38, Loss: 0.4619, Val Loss: 0.4896, Val QWK: 0.2842
Epoch 10/38, Loss: 0.4429, Val Loss: 0.5123, Val QWK: 0.3278
Epoch 11/38, Loss: 0.4556, Val Loss: 0.5330, Val QWK: 0.3372
Epoch 12/38, Loss: 0.4512, Val Loss: 0.5320, Val QWK: 0.3312
Epoch 13/38, Loss: 0.4513, Val Loss: 0.4710, Val QWK: 0.3567
Epoch 14/38, Loss: 0.4528, Val Loss: 0.4919, Val QWK: 0.3343
Epoch 15/38, Loss: 0.4483, Val Loss: 0.4901, Val QWK: 0.3764
Epoch 16/38, Loss: 0.4395, Val Loss: 0.5310, Val QWK: 0.2800
Epoch 17/38, Loss: 0.4453, Val Lo

In [7]:
submission4

,id,sii
0,00008ff9,0
1,000fd460,0
2,00105258,1
3,00115b9f,0
4,0016bb22,1
5,001f3379,0
6,0038ba98,0
7,0068a485,0
8,0069fbed,1
9,0083e397,2
